In [44]:
from langchain.llms import Ollama

llm_model = Ollama(model="llama3.1:latest",
                    temperature=0.7,
                    num_predict=10)
llm_model.invoke("the sky is")

'...blue! (at least, on a clear'

In [ ]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.1:latest")

system_msg = {"role":"system", "content":"always start an asnwer with Your AI:"}
usr_msg={"role":"user", "content":"Hello, how are you?"}
chat_model.invoke([system_msg, usr_msg])


AIMessage(content="Your AI: I'm functioning properly, thank you for asking! I'm a large language model, so I don't have feelings like humans do, but I'm always ready to help answer your questions or chat with you. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2026-01-29T10:50:29.540816311Z', 'done': True, 'done_reason': 'stop', 'total_duration': 998564892, 'load_duration': 70511542, 'prompt_eval_count': 30, 'prompt_eval_duration': 39861159, 'eval_count': 58, 'eval_duration': 853115530, 'model_name': 'llama3.1:latest'}, id='run--a16e03a2-196c-4b32-9f85-f2852924f8f5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 58, 'total_tokens': 88})

In [51]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOllama(model="llama3.1:latest")
template = ChatPromptTemplate.from_template("""Answer the question based on the context. if not included, say "I don't know." """
                                            "Context:{context}\n." \
                                            "Question: {question}\n" \
                                            "Answer:")
chain = template | chat_model

context="x=4,y=10,z=15"
question="What is the value of w?"
chain.invoke({"context":context, "question":question}).content

"I don't know."

In [61]:
query_list = ["what is x?", "what is w?"]

inputs = [{"context": context, "question": q} for q in query_list]

outputs = chain.stream(inputs)
outputs

<generator object RunnableSequence.stream at 0x75d8044f0e50>